In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from sklearn.decomposition import TruncatedSVD
# If nltk stop word is not downloaded
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import nltk,csv,numpy
nltk.download('punkt')
from nltk import sent_tokenize, word_tokenize, pos_tag
sentences=[]
reader = csv.reader(open('/content/FlipkartReview.csv', 'rU'), delimiter= ",",quotechar='|')
for line in reader:
    for field in line:
      sentences.append(field)
print(sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
['Title', 'Review', 'Rating', 'Reviewer_name', 'Very Good', '"I ordered this mobile for my sister. (Online classes)It works well for day to day use.Here is my review after 10 days of use:This mobile is good for you', ' but if you are planing to play games like PUBG', ' COD ect. Don’t buy this Camera- 3/5 camera is disappointing both front and back. Also the front camera is slow as well.Face lock- 3/5 average one ', ' it don’t work in low light. Display- 4/5 not a great display though it’s adjustable.Battery- 4.5/5 it has good battery backup 4000mah but a...READ MORE"', '4.3', 'Flipkart Customer', 'Nice product', '"I really miss the finger print scanner and USB type c port. While I can live with the former', ' the later is a crime in 2020.Apart from that', " everything else is good. I really liked the form factor (it's a small device)", ' although a bit slippery. A 5/5 for the displ

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: 'U' mode is deprecated
  """


In [ ]:
df = pd.DataFrame()
df["documents"] = sentences
df.head()
df['clean_documents'] = df['documents'].str.replace("[^a-zA-Z#]", " ")
df['clean_documents'] = df['clean_documents'].fillna('').apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
df['clean_documents'] = df['clean_documents'].fillna('').apply(lambda x: x.lower())

df.head()

,documents,clean_documents
0,Title,title
1,Review,review
2,Rating,rating
3,Reviewer_name,reviewer name
4,Very Good,very good


In [ ]:
# TF-IDF vector
vectorizer = TfidfVectorizer(stop_words='english', smooth_idf=True)
X = vectorizer.fit_transform(df['clean_documents'])
X.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
# SVD represent documents and terms in vectors 
svd_model = TruncatedSVD(n_components=2, algorithm='randomized', n_iter=100, random_state=122)
lsa = svd_model.fit_transform(X)

In [ ]:
#Documents - Topic vector
pd.options.display.float_format = '{:,.16f}'.format
topic_encoded_df = pd.DataFrame(lsa, columns = ["topic_1", "topic_2"])
topic_encoded_df["documents"] = df['clean_documents']
display(topic_encoded_df[["documents", "topic_1", "topic_2"]])

,documents,topic_1,topic_2
0,title,-0.0000000000000000,-0.0000000000000000
1,review,0.0201398528547108,-0.0091127899841094
2,rating,0.0000000000000000,0.0000000000000000
3,reviewer name,0.0000000000000000,-0.0000000000000000
4,very good,0.6598434309239757,-0.2576301868907325
...,...,...,...
59,abin rajan,0.0000000000000000,0.0000000000000001
60,nice product,0.2560999696119261,0.8807553171953303
61,the camera quality poor compared other brands ...,0.2362705850166001,-0.0292014815307756
62,,0.0000000000000000,-0.0000000000000000


In [ ]:
dictionary = vectorizer.get_feature_names()

In [ ]:
encoding_matrix = pd.DataFrame(svd_model.components_, index = ["topic_1","topic_2"], columns = (dictionary)).T
encoding_matrix

,topic_1,topic_2
abin,0.0000000000000000,0.0000000000000001
abt,0.0045344555536715,-0.0016818761897158
added,0.0115844271259152,-0.0018801947597514
adjustable,0.0422475199872832,-0.0158648759574360
apart,0.0000000000000000,0.0000000000000000
...,...,...
works,0.0347993212191025,-0.0138530266873370
world,0.0173911127604645,-0.0025012061672847
worst,0.0446263031842980,-0.0111231636321836
worth,0.0257252178979597,0.0601314697277614
